In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 105.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.150
    Uninstalling botocore-1.29.150:
      Successfully uninstalled botocore-1.29.150
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.150
    Uninstalling boto3-1.26.150:
      Successfully uninstalled boto3-1.26.150
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.114 requires botocore==1.29.114, but you have botocore 1.29.152 which is incompatible.
awscli 1.27.114 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://

# 在运行下面代码前，请确认已经填入LLM_NAME.
位置：./code/inference.py 第37行

In [22]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_english_m_gpu')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_english_m_gpu')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1-10'
instance_type = 'ml.g4dn.12xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    volume_size_in_gb = 100,
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws:iam::150333911459:role/service-role/AmazonSageMaker-ExecutionRole-20220802T133420
sagemaker-us-east-1-150333911459
dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-1-150333911459/llm_english_m_gpu/assets/model.tar.gz
--------------!

In [28]:
# inputs= {
#     "ask": "你好!"

# }

# response = predictor.predict(inputs)
# print(response["answer"])

inputs= {
    "ask": "hello, who is the most famous football player?"

}

response = predictor.predict(inputs)
print(response["answer"])

hello, who is the most famous football player?
Who is the most famous football player in the world?
Who is the most famous football player in the world right now?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of all time?
Who is the most famous football player in the world of al